## Hipótese 1
### Quais lojas estão com dias de ruptura maior que o leadtime?


In [1]:
import pandas as pd
import numpy as np
import os
import datetime

# Froms
from gcpUtils.auth import getCredentials
from gcpUtils.bigQuery import pandasToBq, tableToPandas
from gcpUtils.google_storage_manager import *

# Definição das credenciais 
cred = getCredentials("../bd/planejamento-animale-292719-296d49ccdea6.json")

## Rupturas em Geral
* Filtramos o banco de dado e tomamos apenas os dados relativos a filiais com CM no final;
* Estabelecemos um período de um ano (facilmente alterável)
* Contamos os dias em que o produto ficou em ruptura, desconsiderando buracos problemáticos, isto é, tomamos por finalização do estado de ruptura o momento em que uma mudança de 1 para 0, mesmo que haja um buraco de dados anterior a isso.
* Ao final, ordenamos por filial e SKU. 

#### Registramos um ID para cada período de ruptura, registrando também a quantidade de dias relativos a cada um deles. Exemplo: se houve $n$ estados de ruptura relativos ao SKU='X', então há $1,2,3,...n$ IDs diferentes, onde cada um indica um estado de ruptura registrado.
#### Os dados então são salvos na pasta "data", em um arquivo xlsx.
#### Note que para especificar um

In [ ]:
query = """
WITH rupturas AS (
    SELECT
        FILIAL,
        SKU,
        DATA,
        CONT_RUPTURA,
        CASE 
            WHEN CONT_RUPTURA = 1
                 AND LAG(CONT_RUPTURA, 1, 0) OVER (PARTITION BY FILIAL, SKU ORDER BY DATA) = 0
            THEN 1 ELSE 0
        END AS inicio_ruptura
    FROM `planejamento-animale-292719.checklists_rollout.ANIMALE_checklist`
    WHERE DATA > '2024-08-01' 
      AND DATA < '2025-09-01'
      AND FILIAL LIKE '%CM%' 
),
grupos AS (
    SELECT
        FILIAL,
        SKU,
        DATA,
        CONT_RUPTURA,
        SUM(INICIO_RUPTURA) OVER (PARTITION BY FILIAL, SKU ORDER BY DATA ROWS UNBOUNDED PRECEDING) AS RUPTURA_ID
    FROM RUPTURAS
    WHERE CONT_RUPTURA = 1
),
resumo AS (
    SELECT
        FILIAL,
        SKU,
        ruptura_id,
        MIN(CAST(DATA AS DATE)) AS inicio_ruptura,
        MAX(CAST(DATA AS DATE)) AS fim_ruptura,
        DATE_DIFF(MAX(CAST(DATA AS DATE)), MIN(CAST(DATA AS DATE)), DAY) + 1 AS dias_ruptura
    FROM grupos
    GROUP BY FILIAL, SKU, ruptura_id
)
SELECT *
FROM resumo
ORDER BY FILIAL, SKU;

"""

In [16]:
df_prod_desc = tableToPandas(query, 'planejamento-animale-292719', cred)
df_prod_desc.columns = df_prod_desc.columns.str.upper()

print(df_prod_desc)

                       FILIAL                    SKU  RUPTURA_ID  \
0       ANIMALE ALPHAVILLE CM  03.01.1448-0005-TAM_2           1   
1       ANIMALE ALPHAVILLE CM  03.01.1448-0005-TAM_3           1   
2       ANIMALE ALPHAVILLE CM  03.01.1469-0005-TAM_2           1   
3       ANIMALE ALPHAVILLE CM  03.01.1469-0005-TAM_2           2   
4       ANIMALE ALPHAVILLE CM  03.01.1469-0005-TAM_3           1   
...                       ...                    ...         ...   
563059              BUZIOS CM  52.57.0013-6206-TAM_3           1   
563060              BUZIOS CM  52.N8.0004-8259-TAM_4           1   
563061              BUZIOS CM  52.N8.0004-8259-TAM_4           2   
563062              BUZIOS CM  73.01.0071-0003-TAM_1           1   
563063              BUZIOS CM  73.01.0071-0003-TAM_2           1   

       INICIO_RUPTURA FIM_RUPTURA  DIAS_RUPTURA  
0          2024-12-30  2025-01-28            30  
1          2024-12-30  2025-01-28            30  
2          2024-12-30  2024-12-30

In [19]:
# Salvando o dataframe em um arquivo excell localmente
local_path = os.path.join(os.getcwd(), 'data')
if not os.path.exists(local_path):
    os.makedirs(local_path)
file_path = os.path.join(local_path, 'rupturas_anuale_checklist.xlsx')
df_prod_desc.to_excel(file_path, index=False)
